
# 执行reshape操作

例如 , reshaped = torch.reshape(input_tensor, (-1, 1, 2, 2))

        (-1, 1, 2, 2) 表示：

        -1：自动计算批量大小

        1：通道数为1（单通道）

        2：高度为2

        2：宽度为2

        场景 :
            1. 转换为CNN可接受的格式（批量×通道×高度×宽度）
            2.转换为批量处理格式 （批量×特征）
                    # 如果有多个2×2的图像块
                    patches = [
                        [[1, 2], [-1, 3]],    # 第一个patch
                        [[4, 5], [6, 7]],     # 第二个patch
                        [[8, 9], [10, 11]]    # 第三个patch
                    ]

                #意思是3个2×2的图像块，每个图像块有1个通道
                batch_patches = torch.reshape(torch.tensor(patches), (-1, 1, 2, 2))
                print("批量patch形状:", batch_patches.shape)  # torch.Size([3, 1, 2, 2])

            3.  ReLU激活函数




例如 , reshaped = torch.reshape(input_tensor, (1, 1, 1, -1))

        (1, 1, 1, -1) 表示：

        1：批量大小固定为 1（单样本输入）

        1：通道数为 1，方便复用卷积/归一化等层

        1：高度设为 1，仅占位无需真实二维结构

        -1：让 PyTorch 自动推断剩余长度作为宽度，保持元素总数不变
        (也就是说 , 把向量展平成 1×1×1×n 的形式),
        (-1替换为别的数字, 就是指 一行有 n 个元素)

        场景 :
            1. 将任意向量快速包装成 4 维形状（批量×通道×高度×宽度），复用现有 CNN/PyTorch API。
            2. 临时对齐模型接口：某些层要求输入至少四维，可用 1 作为占位维度。
            3. 调试或可视化时，把长向量“铺平”到最后一维，便于打印或绘图。

        示例 :
            import torch

            input_tensor = torch.arange(6)# 生成一个包含6个元素的向量

            reshaped = torch.reshape(input_tensor, (1, 1, 1, -1)) # 和flatten()效果相同
            print('原始形状:', input_tensor.shape)  # torch.Size([6])
            print('新形状:', reshaped.shape)       # torch.Size([1, 1, 1, 6]),化为1×1×1×6的4维张量
            print('数据保持不变:', reshaped)

        理解:
              - 如果你的下一层是线性层或任何接受二维 (batch, features) 输入的模块，直接 flatten()
                就行；再 reshape(1, 1, 1, -1) 只是多绕了一圈。
              - 只有当下一层依赖 4 维输入（如你想临时把向量喂给卷积层试验）时，才会用 (1, 1, 1, -1)
                这种“伪”4D 包装。

              总结：常规分类流程里 flatten() 已满足需求，无需再 reshape(1, 1, 1, -1)；两者都能保留
              元素顺序，但面向的下游接口不同。

        例子 :
                  如果把规模缩小到 Linear(4, 2) 会更直观：

                  权重 W =
                  [[w_11, w_12, w_13, w_14],   ← 输出神经元 1 的 4 个权重
                   [w_21, w_22, w_23, w_24]]   ← 输出神经元 2 的 4 个权重

                  输入向量 x = [x1, x2, x3, x4]，那么：

                  logit_1 = w_11*x1 + w_12*x2 + w_13*x3 + w_14*x4 + b_1
                  logit_2 = w_21*x1 + w_22*x2 + w_23*x3 + w_24*x4 + b_2

                  . Linear(4096, 512) 做了什么？

                  - 这层本质是一次矩阵乘法：y = W_hidden x + b_hidden。
                  - W_hidden 的形状是 512 × 4096，也就是 512 行、4096 列。
                  - b_hidden 是长度 512 的偏置向量。
        于是 4096 维向量被“压缩/组合”成 512 维新的特征向量 y。这一层允许网络学到比卷积层更灵活、更高阶的特征组合

In [9]:

# 理解argmax函数
import torch
# 示例张量
logits = torch.tensor([[1.0, 2.0, 3.0],  # 第一个样本的 logit 向量
                        [2.0, 1.0, 3.0]]) # 第二个样本的 logit 向量


# 对每个样本的 logit 向量应用 argmax
#  dim=1 表示在每个样本的 logit 向量上应用 argmax，得到每个样本的预测类别
# 解释 :
#  第一个样本的 logit 向量为 [1.0, 2.0, 3.0]，最大元素是 3.0，对应索引 2。
#  第二个样本的 logit 向量为 [2.0, 1.0, 3.0]，最大元素是 3.0，对应索引 2。
#  所以，argmax 函数返回的预测类别为 [2, 2]，表示第一个样本预测为类别 2，第二个样本也预测为类别 2。
#
#  dim=0 表示在每个特征维度上应用 argmax，得到每个特征的最大元素索引
# 解释 :
#  第一个样本的 logit 向量为 [1.0, 2.0, 3.0]，最大元素是 3.0，对应索引 2。
#  第二个样本的 logit 向量为 [2.0, 1.0, 3.0]，最大元素是 3.0，对应索引 2。
#  所以，argmax 函数返回的预测类别为 [2, 2]，表示第一个样本预测为类别 2，第二个样本也预测为类别 2。
predicted_classes = torch.argmax(logits, dim=1)
print("预测类别:", predicted_classes)